In [40]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.common.by import By
import pandas as pd

### Let's do web scrapping of afisha.ru: we need to collect info about restaurants in Moscow

#### Firstly, we'll collect the restaurants links by going through all the pages with Selenium.

In [50]:
base_url = "https://www.afisha.ru"
url = f"{base_url}/msk/restaurants/restaurant_list/"

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

driver.get(url)
time.sleep(7)

all_links = set()  #We need to use set to avoid repeat links

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for a in soup.find_all("a"):
        if "href" in a.attrs and a["href"].startswith("/msk/restaurant"): #Search only links starts with /msk/restaurant
            all_links.add(a["href"])

    try:
        next_button = driver.find_element(By.CLASS_NAME, 'Pagination_pagination-next__rtqsZ')#try to click next page button
        next_button.click()
        time.sleep(5)
    except Exception as e:
        try:
            close_button = driver.find_element(By.CLASS_NAME, 'popmechanic-close')#unexpectedly a banner pops up on the site, try to close it, as it covers the usual page layout
            close_button.click()
            time.sleep(2)

            next_button = driver.find_element(By.CLASS_NAME, 'Pagination_pagination-next__rtqsZ')#try to click next page button again
            next_button.click()
            time.sleep(5)
        except Exception as inner_e:
            print("There are no more pages:", inner_e)
            break

driver.quit()

Python(7636) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(7677) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


KeyboardInterrupt: 

In [52]:
#adding base url to links
full_links = [base_url + link for link in all_links]
all_links_list = list(full_links)

df = pd.DataFrame(all_links_list, columns=['Link'])
df.to_csv('resto_links.csv', index=False)

print(f"We saved {len(all_links_list)} links in 'rest_links.csv'")

We saved 11957 links in 'rest_links.csv'


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import csv
import time
from selenium.webdriver.common.by import By
import pandas as pd


with open('data.csv', 'w', encoding='utf-8-sig', newline='') as d:
    file_writer =  csv.writer(d, delimiter = ";")
    file_writer.writerow(["название ресторана", "оценка","адрес", "метро", "средний чек", "завтраки","бизнес-ланч", "доставка","парковка","кейтеринг","банкеты","телефон", "сайт", "тип заведения","открыто"]) 

with open('resto_links.csv','r') as f:
    all_links = f.readlines()
for link in all_links:

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    driver.get(link)
    time.sleep(7)

    soup = BeautifulSoup(driver.page_source, "html.parser")

        
    name = soup.find('span',class_ = "Title_header__SIloF").text
    #bill = soup.find('div',class_ = "RestaurantExtraInfo_table__l34_J").find('div',class_ = "RestaurantExtraInfo_table-cell__fk6SJ").find('div',class_ = "RestaurantExtraInfo_cell-data__3Xfkk").text
    rank = soup.find('div',class_ = "RestaurantCover_rating-wrapper__CTNts")
    if rank!=None:
        estimate = rank.text
    else:
        estimate = 0
    adress = soup.find('div', class_="SectionTitle_wrapper__nAAJ0 RestaurantExtraInfo_address__aJsK2").text
    metro = soup.find_all('ul', class_="RestaurantExtraInfo_metro-list__KTBX3")
    metro_=[]
    for i in metro:
        for j in i.find_all('span', class_="Text_text__e9ILn"):
            metro_ += [j.text]
    s = ''       
    all = soup.find_all('div',class_ = "RestaurantExtraInfo_table__l34_J")
    for i in all:
        for j in i.find_all('span', class_="Text_text__e9ILn"):
           s += j.text + ','
        tot =s.split(',')[1:len(s):2]   
    bill = tot[0] 
    brekfast = tot[1]
    business =  tot[2]
    delivery = tot[3]
    parking = tot[4]
    keit = tot[5]
    feast = tot[6] 
    phone_num = tot[7]
    site_ = tot[8] 
    type_all = []
    type_ = soup.find_all('div',class_ = "RestaurantExtraInfo_tag__BqQ7e")
    for i in type_:
        for j in i.find_all('div', class_="Button_button__content___D2b_"):
            type_all +=[j.text] 
    work = soup.find_all('div',class_ = "RestaurantCover_content-wrapper__72Dox")
    for i in work:
        for j in i.find_all('span', class_="Text_text__e9ILn"):
            e = j          
    open_ = e.text.strip('Открыто c')    

    #good = soup.find_all('div',class_ = "Button_button__content___D2b_")
    #print(link)
    #for i in good:
        #for j in i.find_all('span', class_="DefaultTag_button__counter__64UpQ"): 
            #good1 = j.text
                  
    with open('data.csv', 'a', encoding='utf-8-sig', newline='') as d:
        file_writer =  csv.writer(d, delimiter = ";")
        file_writer.writerow([name,estimate,  adress, metro_, bill, brekfast ,business ,delivery ,parking ,keit ,feast ,phone_num , site_, type_all, open_])